In [7]:
from IPython.display import HTML
HTML('''
<script>
    // AUTORUN ALL CELLS ON NOTEBOOK-LOAD!
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
            });
        }
    );
</script>
''')

In [8]:
HTML('''
<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>
''')

# Eye Diagrams
<p style="font-size:18px">
This notebook demonstrates how increased insertion loss and decreased extinction ratio for an optical modulator leads to increased errors in the reading of a bits in an optical data stream. The notebook also introduces the concept of an eye diagram, and illustrates how eye diagrams can be used distinguish between sources of bit errors such as insertion loss and timing jitter.
<p style="font-size:18px">  
Launch this notebook by selecting "Run All" under the "Cell" menu. You can view the python code used to create data for the plots by clicking on the "Show Code" button just below.

In [6]:
from ipywidgets import interact, interactive, FloatSlider, IntSlider, Button, ToggleButtons, HBox, VBox, Label
from IPython.display import display
import numpy as np
from numpy import fft
from scipy import special, signal
from bokeh.io import push_notebook, curdoc
from bokeh.themes import Theme
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, Range1d, LinearAxis, Title, Label, Span, Band
from bokeh.layouts import column, row
import yaml
import warnings; warnings.simplefilter('ignore')
output_notebook()

curdoc().theme = Theme(json=yaml.load("""
    attrs:
        Figure:
            background_fill_color: "lightgray"
            outline_line_color: black
            height: 400
            width: 400
        Grid:
            grid_line_color: white
            grid_line_alpha: 0.5
        Axis:
            axis_label_text_font_size: "12pt"
            axis_label_text_font_style: "normal"
            major_label_text_font_size: "12pt"
            major_label_text_font_style: "normal"
        Line:
            line_alpha: 1
            line_width: 1.5
        Title:
            text_font_style: "normal"
            text_font_size: "12pt"
        Label:
            text_color: "darkblue"
            background_fill_color: "lightgray"
"""))

Loading BokehJS ...

## Amplitude Noise, Q-Factor, and Bit Error Rate
<p style="font-size:18px">
One of the most important contributions to errors in the detection of optical signals is thermal noise in the optical receiver. Thermal noise arises from fluctuations in the current through a resitive load. It has a Gaussian distribution that is characterized by the standard deviation for the noise current, given by the expression
<p style="font-size:18px">
$${\sigma _T} = \sqrt {\frac{{4{k_B}T}}{{{R_L}}}\Delta f}, $$
where $k_B$ is Boltzman's constant, T is the temperature, $R_L$ is the resistance of the load, and $\Delta f$ is the bandwidth of the receiver. Current variation leads to a variation in the voltage across the resisttor and and to a variation in the amplitude of the measured signal.
<p style="font-size:18px">
Thermal noise constibutes equally to variation in the amplitude for transmitted "ones" and "zeros". More generally there are additional noise sources that contibute in different amounts to ones and zeros (shot noise is an example), and that are characterized by two different standard deviations ${\sigma _1}$ and ${\sigma _0}$.
<p style="font-size:18px">
A quantity called the Q-Factor is the generalization for signal-to-noise-ratio that is used when the amount of noise depends on signal level.
<p style="font-size:18px">
$$Q \equiv \frac{{{I_1} - {I_0}}}{{{\sigma _0} + {\sigma _1}}},$$
<p style="font-size:18px">
where $I_1$ and $I_0$ are the average current generated by a transmitted one and zero, respectively. The bit error rate (BER, the fraction of bits incorrectly detected) is related to Q-Factor  by the expression
<p style="font-size:18px">
$$BER = \frac{1}{2}erfc\left( {\frac{Q}{{\sqrt 2 }}} \right),$$
where erfc is the complementary error function.

### Exploration
<p style="font-size:18px">
Explore how increased insertion loss and decreased extinction ratio signal nois contibute to the Q-Factor and bit-error rate with the interactive graphs below. The graph on the left below shows a signal that represents the bit sequence 1,0,1,0,1. The graph on the right below shows the theoretical distributions of values of the signal when a zero is transmitted (blue curve) and when a one is transmitted (orange curve). A transmitted bit is determined to the a one if a measurment is above the voltage threshold, and a zero if a measurement is below the threshold. Signals are measured at the middle of a bit period. There is always a non-zero probability that a signal will be on the wrong side of the threshold level during a measurment. These values are represented by the overlap of the shaded regions under the blue and orange curves.
<p style="font-size:18px">
Suggestion: Adjust the average received power and noise levels to achieve a bit error rate of 1 x $10^{-12}$. Note the corresponding Q-Factor.
<p style="font-size:18px">
Suggestion: Adjust the sliders so that the signal level for a one occasionally falls below the threshold level, and examine the corresponding Q-Factor and bit error rate.n

In [4]:
R = 0.5 # Responsivity
RL = 50 # Load Resistor, Ohms
G = 100 # Post Load Amplifier Gain
KB = 1.38*10**-23 ## Boltzmann constant MKS units
delta_f = 1250 * 10**9
T = 300 # Temperature, Degrees Kelvin
B = 10e9 # Bit Rate in gigabits per second
NF = 3 # Post photodiode amplifier noise figure of 5 dB
filter_bandwidth=10e9
sigma_th_BWL = np.sqrt(KB*T*2*filter_bandwidth/RL) # Bandwidth limited thermal noise, bandwidt equal to bit rate B
Pavemax = 36e-6 # Average power in Watts without modulator insertion loss

# *** Set up plot p ***
p = figure(title='Signal with Amplitude Noise')
p.min_border_top = 10
p.x_range = Range1d(0, 500)
p.y_range = Range1d(-25.0, 120.0)
p.xaxis.axis_label="Time (psec)"
p.yaxis.axis_label="Voltage (mV)"

time_window = 500 # time window in psec
num_samples = 5000 # number of samples in window
t=np.linspace(0,time_window,num_samples)
sigma_th = np.sqrt(KB*T*1e12*(num_samples)/time_window/2*NF/RL) # Thermal noise with contributions up to the Nyquist frequency
zeros=np.zeros(num_samples)
v=zeros
Threshold=zeros
fft_freq = np.linspace(0,1000*num_samples/time_window/2,np.int(num_samples/2+1))
b, a = signal.bessel(4, filter_bandwidth*1e-9, 'low', analog=True, norm='phase')
w, h = signal.freqs(b, a, fft_freq)
filter=np.absolute(h)

l=p.line(t,v,line_color='#2222aa')
lt=p.line(t,Threshold,line_dash='dashed',line_color='red')

label=Label(x=320, y=110) # Q-Factor Label
label1=Label(x=10, text='V Threshold')
p.add_layout(label); p.add_layout(label1)
one_label_1=Label(x=28, y=100,text='one') # Bit Label
p.add_layout(one_label_1)
zero_label_1=Label(x=123, y=100,text='zero') # Bit Label
p.add_layout(zero_label_1)
one_label_2=Label(x=228, y=100,text='one') # Bit Label
p.add_layout(one_label_2)
zero_label_2=Label(x=323, y=100,text='zero') # Bit Label
p.add_layout(zero_label_2)
one_label_3=Label(x=428, y=100,text='one') # Bit Label
p.add_layout(one_label_3)

# *** Set up plot p1 ***
p1 = figure(y_axis_type="log", title='Noise Distribution')
p1.x_range = Range1d(-70, 170)
p1.y_range = Range1d(1e-16, 40)
p1.xaxis.axis_label="Voltage (mV)"
p1.yaxis.axis_label="Probabilty"

vline = Span(dimension='height', line_color='red', line_dash='dashed')
p1.add_layout(vline)
vvalue=np.linspace(-80,180,1000)
band_source = ColumnDataSource(data=dict(
    x = vvalue,
    ylow = np.ones(1000)*1e-16,
    yup1 =  np.ones(1000)*1e-16,
    yup2 =  np.ones(1000)*1e-16
))
band = Band(base='x', lower='ylow', upper='yup1', source=band_source, fill_alpha=0.5, line_width=2,
            fill_color='blue', line_color='darkblue', line_alpha=1.0)
p1.add_layout(band)
band2 = Band(base='x', lower='ylow', upper='yup2', source=band_source, fill_alpha=0.5, line_width=2,
            fill_color='yellow', line_color='orange', line_alpha=1.0)
p1.add_layout(band2)

label2=Label(x=77,  y=1.5, text='BER=')
label3=Label(x=16*R*RL*G*1000+4, y=0.05, text='V Threshold')
p1.add_layout(label2); p1.add_layout(label3)


# *** Define sliders ***
slider_style = {'description_width': 'initial'}
Ins_Loss_slider=FloatSlider(min=3.0, max=13.0, step=0.1, value=3.0, description='Insertion Loss (dB)',
                            style=slider_style, continuous_update=True)
ext_ratio_slider=FloatSlider(min=0, max=30.0, step=0.5, value = 30.0, description='Extinction Ratio', continuous_update=True,
                            style = {'description_width': '108px'}, layout={'width': '315px'})

def replot(Ins_Loss,ext_ratio):
    # Compute new signal
    Pave=Pavemax*10**(-Ins_Loss/10)
    ext_ratio=1-10**(-ext_ratio/10)
    v0 = np.random.normal(Pave*2*R*RL*G*1000, sigma_th*RL*G*1000, 1001)
    v1 = np.random.normal((1-ext_ratio)*Pave*2*R*RL*G*1000, sigma_th*RL*G*1000, 1000)
    v2 = np.random.normal(Pave*2*R*RL*G*1000, sigma_th*RL*G*1000, 1000)
    v3 = np.random.normal((1-ext_ratio)*Pave*2*R*RL*G*1000, sigma_th*RL*G*1000, 1000)
    v4 = np.random.normal(Pave*2*R*RL*G*1000, sigma_th*RL*G*1000, 1000)
    v = np.concatenate([v0,v1,v2,v3,v4])
    
    sig_fft= np.fft.rfft(v)
    sig_fft_f = sig_fft*filter
    inv_sig_fft=np.fft.irfft(sig_fft_f)
    
    # Update lines and bands
    lt.data_source.data['y']=zeros+Pave*R*RL*G*1000*(2-ext_ratio)
    l.data_source.data['y']=inv_sig_fft
    vline.location=Pave*R*RL*G*1000*(2-ext_ratio)
    band_source.data['yup1']=1/(sigma_th_BWL*RL*G*1000*(2*np.pi)**0.5)*np.exp(-(vvalue-2*Pave*R*RL*G*1000*(1-ext_ratio))**2
                                        /(2*(sigma_th_BWL*RL*G*1000)**2))
    band_source.data['yup2']=1/(sigma_th_BWL*RL*G*1000*(2*np.pi)**0.5)*np.exp(-(vvalue-2*Pave*R*RL*G*1000)**2
                                        /(2*(sigma_th_BWL*RL*G*1000)**2))
    
    # Update labels and push changes to web browser
    Q = (2*Pave*R*ext_ratio)/(sigma_th_BWL+sigma_th_BWL); label.text='Q-Factor='+np.str(Q)[:4]
    label1.y=Pave*R*RL*G*1000*(2-ext_ratio)
    BER = 0.5*special.erfc(Q/np.sqrt(2))
    label2.text='BER='+'{:.2e}'.format(BER)
    label3.x=Pave*R*RL*G*1000*(2-ext_ratio)+4
    push_notebook(handle=fig_handle)

fig_handle = show(row(p,p1), notebook_handle=True)
interact(replot, Ins_Loss=Ins_Loss_slider, ext_ratio=ext_ratio_slider);

interactive(children=(FloatSlider(value=3.0, description='Insertion Loss (dB)', max=13.0, min=3.0, style=Slide…

## Building an Eye Diagram
<p style="font-size:18px">
A measurment of bit error rate provides a single, quantitative value that provides a valuable, overall assessment of the peformance of an optical link. An eye diagram is a visual tool that can be used to quickly produce a qualtitative assessment of the various contributions to the bit error rate, including themal noise and timing jitter. An eye diagram is created by overlapping multiple signal traces, each carrying a separate random bit pattern. The traces are typically two bit periods long, with half a bit period followed by a full bit period, followed by half a bit period.

### Timing Jitter
<p style="font-size:18px">
Jitter refers to a shift of a received signal relative to a sampling time. A small amount of jitter would not result in bit erors if it were not for the finite bandwidth of the receiver, which rounds singals during a bit period. Signals are typically peaked at the center of a bit period, where sampling takes place, and jitter shifts the peak relative to the sampling time.

### Exploration
<p style="font-size:18px">
The graph just below shows how an eye diagram is built up by drawing multiple segments of bit stream. Click on the "Trace" button to place a signal curve on the graph. The random bit pattern will be one of eight - 000. 001, 010, 011, 100, 101, 110, or 111. Press the Trace button repeatly to place multiple signal curves on the graph. (Up to 21 curves can be simultaneously drawn on the graph.) The "Reset" button clears the graph. Eliminate timing jitter from the signal by clicking the button labeled "0 psec" and and observe the qualtative change in the eye diagram.

<p style="font-size:18px">
Suggestion: Find the minimum number of signal traces needed to create an eye diagram that shows both the vertical and horizontal undertainty in the signal values.

In [5]:
jitter_toggle = ToggleButtons(options=['0 psec','10 psec'],value='10 psec',description='Jitter Value')
trace_button = Button(description="Trace")
reset_button = Button(description="Reset")
trace_button.style.button_color = 'lightgreen'; reset_button.style.button_color = 'lightgreen'

display(VBox([HBox([trace_button, reset_button]),jitter_toggle]))
    
p4 = figure(title='Q ='+'  BER=')
p4.min_border_top = 10
p4.x_range = Range1d(150, 350); p4.y_range = Range1d(-25.0,105.00)

line_list=[]
for i in range(21):
    line_list.append(p4.line(t,zeros-2,line_color='#2222aa'))

p4.xaxis.axis_label="Time (psec)"
p4.yaxis.axis_label="Voltage (mV)"

Pave = 18e-6 # Average signal power in watts

def update_plot(v):
    global count
    global for_std
    
    v = np.random.normal(np.random.randint(0, 2)*Pave*2*R*RL*G, sigma_th*RL*G, 1000)*1000
    for i in range (4):
        v = np.append(v,np.random.normal(np.random.randint(0, 2)*Pave*2*R*RL*G, sigma_th*RL*G, 1000)*1000)
    
    if jitter_toggle.value == '0 psec':
        jitter=0
    else:
        jitter=10
    tj = t - np.random.normal(0,jitter)
    sig_fft= np.fft.rfft(v)
    sig_fft_f = sig_fft*filter
    vf=np.fft.irfft(sig_fft_f)
    
    line_list[count].visible=True
    line_list[count].data_source.data['x']=tj
    line_list[count].data_source.data['y']=vf

    count += 1
    if count == 21:
        count = 0
    push_notebook(handle=fig_handle3)
    
fig_handle3 = show(p4, notebook_handle=True)
trace_button.on_click(update_plot)

def initialize_plot(pa):
    global count
    for i in range(21):
        line_list[i].visible=False
    count = 0
    push_notebook(handle=fig_handle3)

initialize_plot(p4)

def calc_Q_and_BER(pa):
    initialize_plot(p4)
    if jitter_toggle.value=='0 psec':
        jitter=0
    else:
        jitter=10
    zero_values=[]
    one_values=[]
    for i in range(3000):
        ranint = np.random.randint(0,2)
        v0 = np.random.normal(np.random.randint(0, 2)*Pave*2*R*RL*G, sigma_th*RL*G, 1000)
        v1 = np.random.normal(np.random.randint(0, 2)*Pave*2*R*RL*G, sigma_th*RL*G, 1000)
        v2= np.random.normal(ranint*Pave*2*R*RL*G, sigma_th*RL*G, 1000)
        v3= np.random.normal(np.random.randint(0, 2)*Pave*2*R*RL*G, sigma_th*RL*G, 1000)
        v4= np.random.normal(np.random.randint(0, 2)*Pave*2*R*RL*G, sigma_th*RL*G, 1000)
        v = np.concatenate([v0,v1,v2,v3,v4])*1000
        index_shift = round(np.random.normal(0,jitter)*10)

        sig_fft= np.fft.rfft(v)
        sig_fft_f = sig_fft*filter
        vf=np.fft.irfft(sig_fft_f)
        if ranint == 0:
            zero_values=np.append(zero_values,vf[2500-index_shift])
        else:
            one_values=np.append(one_values,vf[2500-index_shift])
    zero_mean=np.mean(zero_values)
    one_mean=np.mean(one_values)
    zero_std=np.std(zero_values-zero_mean)
    one_std=np.std(one_values-one_mean)
    Q = (one_mean-zero_mean)/(zero_std+one_std)
    BER = 0.5*special.erfc(Q/np.sqrt(2))
    p4.title.text='Q ='+'{:2.1f}'.format(Q)+'  BER='+'{:.2e}'.format(BER)
    push_notebook(handle=fig_handle3)
calc_Q_and_BER(p4)


jitter_toggle.observe(calc_Q_and_BER,names='value')
reset_button.on_click(initialize_plot)